In [11]:
#!/usr/bin/env python
# coding: utf-8
from Bio import Entrez, SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import numpy as np
import pandas as pd

In [3]:
def get_dna_sequence(chr_id,start,stop, stran):
    Entrez.email = "yuhonguni@gmail.com"  #always tells NCBI who you are
    handle = Entrez.efetch(db="nucleotide", 
                           id=chr_id, 
                           rettype="fasta", 
                           strand=stran, 
                           seq_start=start, 
                           seq_stop=stop)
    record = SeqIO.read(handle, "fasta")
    handle.close()
    return(record)

In [4]:
human_chr_ID = {'chr1':'NC_000001.11',
               'chr2':'NC_000002.12',
               'chr3':'NC_000003.12',
               'chr4':'NC_000004.12',
               'chr5':'NC_000005.10',
               'chr6':'NC_000006.12',
               'chr7':'NC_000007.14',
               'chr8':'NC_000008.11',
               'chr9':'NC_000009.12',
               'chr10':'NC_000010.11',
               'chr11':'NC_000011.10',
               'chr12':'NC_000012.12',
               'chr13':'NC_000013.11',
               'chr14':'NC_000014.09',
               'chr15':'NC_000015.10',
               'chr16':'NC_000016.10',
               'chr17':'NC_000017.11',
               'chr18':'NC_000018.10',
               'chr19':'NC_000019.10',
               'chr20':'NC_000020.11',
               'chr21':'NC_000021.09',
               'chr22':'NC_000022.11',
               'chrX':'NC_000023.11',
               'chrY':'NC_000024.10'}

In [5]:
#transform the consensus sequences to instance sequences

def append_all(s1, s2):  # append all elements in s2 to each element in s1 respectively. 
    s3 = []
    if len(s1) == 0:  # special case: s1 is empty, then len(s1) is 0
        s3 = s2
    else:
        for i in range(0, len(s1)):
            for j in range(0, len(s2)):
                s3.append(s1[i] + s2[j])
    return(s3)

def consensus_to_instances(consensus_seq):
    dict_decode = {'A':['A'], 
                   'T':['T'], 
                   'C':['C'], 
                   'G':['G'],
                  'R': ['G', 'A'],
                  'Y': ['T', 'C'],
                  'M': ['A', 'C'],
                  'K': ['G', 'T'],
                   'S': ['G', 'C'],
                   'W': ['A', 'T'],
                   'H': ['A', 'C', 'T'],
                   'B': ['G', 'T', 'C'],
                   'V': ['G', 'C', 'A'],
                   'D': ['G', 'A', 'T'],
                   'N': ['G', 'A', 'T', 'C']}
    motif_instances = []
    for motif in consensus_seq:  # loop each letter in consensus_seq (named as 'motif')
        instances = dict_decode[motif]  # get the corresponding instance letters
        motif_instances = append_all(motif_instances, instances)  # append all instances to the current motif_instances
    return(motif_instances)

In [1]:
# input RNA regulatory motif information 
as_motif = pd.read_csv('/home/yu/python_script/AS_motif.csv',sep = ';')
i = 1
while i < len(as_motif):
    if np.isnan(as_motif.iloc[i,0]):
        as_motif.loc[i-1,'Sequence'] = as_motif.loc[i-1,'Sequence'] + as_motif.loc[i,'Sequence']
        as_motif.drop(i,axis = 0,inplace=True)
        as_motif.reset_index(drop = True,inplace=True)      
    else:
        i = i + 1

NameError: name 'pd' is not defined

In [10]:
#Transform all the consensus sequences to instances and stored in a dictionary
motif_dic = dict()
for i in range(0,len(as_motif)):
    a = str(as_motif.loc[i,'AEDBMotif ID'])
    b = as_motif.loc[i,'Type']
    c = as_motif.loc[i,'Organism']
    d = as_motif.loc[i,'Sequence']
    c = a + '-' + b + '-' + c + '-' + d
    motif_dic[c] = consensus_to_instances(d)